# Entropy, conditional entropy, and mutual information

Overview:
* In this tutorial, we will learn how to describe the information that is shared between two variables (mutual information). In other words, how much uncertainty reduction is there to be had about variable 1 by measuring variable 2? 

* These concepts were initially developed in communication theory to describe the efficacy of transmitting signals over a noisy medium (like a noisy telephone line). For example, suppose that we want to know how good a communication channel is, or its effeciency in reliably relaying a message from point A (a 'sender') to point B (a 'reciever').

* Basically this is just like asking, "we know how good the signal is at A, and we recieved the message at B - how much information about A is still in the received signal B?". 

* So that is the general gist of it, but right away you can see the potential applicability of this metric in many fields of neuroscience, psychology, engeneering, etc. In neuroscience, we're bascially dealing with a series of communication channels that are corrupted by noise (i.e. synapses). It is therefore reasonable to ask: how much information from neuron A effectively propogates to neuron B? (or conversely, how much  information is lost?).

* However, this logic works for any combination of variables: two continuous variables, two discrete variables, one continuous and one discrete, etc. As a result, we can ask questions about any two variables really: how much information about median home  price is reflected in stock market fluctuations? etc.

* A few notes before we get started. First, we're going to be talking a lot about 'uncertainty' and 'uncertainty reduction'. While this is basically complementary to talking about certainty and an increase in certainty, we'll deal with the former terminology as it is embedded in some of the concepts that we'll discuss. 

* Second, we'll be dealing with variability in data, and how we can either attribute that variability in the data to 'noise' or to 'signals'. I.e. is the variability in one variable random wrt another variable? or does the variability in one variable systematically change with the variability in another? 

* In addition, MI has has a very intuitive interpretation in terms of the amount of information that is shared between two variables, and we'll get to that in a few minutes. 

<div class="alert alert-info">
Finally, a lot of people think at this point, "why not just correlate the variables using a normal r-value?". There are a few answers to this, but the simplest is this: correlation assumes a linear relationship (or, in more complex forms, a known relationship or you have to assume a relationship) between variables. Mutual information does not, and can generally capture any form of linear or non-linear relationship between two variables. This makes it a very powerful and general purpose metric.
</div>

[jackknife correction](https://www.pnas.org/content/115/40/9956)

<div class="alert alert-warning">
From the above linked paper from Zeng, Xia, and Tong (their Abstract): Quantifying the dependence between two random variables is a fundamental issue in data analysis, and thus many measures have been proposed. Recent studies have focused on the renowned mutual information (MI) [Reshef DN, et al. (2011) Science 334:1518–1524]. However, “Unfortunately, reliably estimating mutual information from finite continuous data remains a significant and unresolved problem” [Kinney JB, Atwal GS (2014) Proc Natl Acad Sci USA 111:3354–3359]. In this paper, we examine the kernel estimation of MI and show that the bandwidths involved should be equalized. We consider a jackknife version of the kernel estimate with equalized bandwidth and allow the bandwidth to vary over an interval. We estimate the MI by the largest value among these kernel estimates and establish the associated theoretical underpinnings.
</div>

* This is a very important concept to deal with - MI, esp for continuous variables, is highly unstable and requires correction procedures to counter the bias that is inherent in estimating MI for small data sets. 
* So while at the start of the tutorial we'll use discrete arrays of numbers that have few unique entries to demonstrate the basic concepts (like binary weighted coin flips, for example), things will get a bit crazier when we move on to continuous arrays of values. 
* Note also that the proposed jacknife correction from this PNAS paper is just one approach...I'm implementing it because in the few cases I've tried, it seems to be pretty numerically stable. Hwoever, there are other slightly more straightforward approaches that build on the ideas that we discussed in the "randomization" tutorial a few weeks back (and indeed,the jacknife approach is logically related as well).  

## Imports

In [2]:
import numpy as np

from scipy.special import gamma,psi
from scipy import ndimage
from scipy.linalg import det
from numpy import pi
import matplotlib.pyplot as plt


In [ ]:
%% PART II: Entropy as a measure of variability
% Shannon Entropy is related to the variability of
% data, but is more specifically defined as the average uncertainty in a
% variable. Consider coin tosses - lets say we have a balanced coin, and we
% flip it once. We can represent the outcome of a single toss as a 0 or a 1
% (a head or a tail), or an entropy of 1 bit. If we flipped the coin twice, 
% then we would have an entropy of 2 bits (00, 01, 10, or 11). When the
% coin is balanced and each outcome is equally likely, then entropy is
% highest and flipping the coin will convey 1 bit of information. 
% To see why entropy is maximized in this situation, consider a biased coin 
% that comes up head 60% of the time. We could predict the outcome of the coin flip more often than
% chance simply by going with our prior of 'head'. Thus, the entropy of the
% biased coin flip is less than the entropy of the unbiased coin flip, as
% the reduction in our uncertainty about the outcome is lower with the
% biased coin than with the balanced coin. Put another way, we learn less
% after flipping the biased coin than we do when we flip the unbiased
% coin. 
% These examples bring up an important point: entropy as a measure of the
% unpredictability of information (or unpredictability of an outcome) is
% maximized when all possible outcomes are equally likely. In this case,
% making a measurement will carry a lot of information, as you have no
% priors upon which to base a guess in advance. In contrast, when only a
% subset of all possible outcomes are actually possible, then entropy
% is lower as the information gain from making a measurement is not much
% beyond your priors (what you already knew).

% to figure out the entropy of a measurement in terms of bits (the most common
% metric), we can use the log2 function,
% referred to as the binary logarithm and the inverse function of 2^N. The
% log2(n) is the power to which the number 2 must be raised to obtain the
% value n. Lets go back to our coin flip example with a fair coin. Suppose
% you flipped the coin once - the entropy would be:

log2(2) % log2(n) or 2^x = n? ...x = 1

% because there are two possible outcomes. 

% if you flipped the coin twice you'd have 4 possible outcomes (00,10,01,11)
%, or three times you'd have 8 possible outcomes:

log2(4)  % or 2^x = n, x = 2
log2(8)  % or 2^x = n, x = 3

In [ ]:
% to see the general shape of the function, plot out log2(x:y) and that will tell you the
% relationship between the number of possible outcomes and the entropy in
% bits
clear 
close all
figure(1)
plot(1:15, log2(1:15), 'LineWidth', 2)
set(gca, 'YLim', [-1,5])
ylabel('Entropy (2^N = ?)')
xlabel('# of possible outcomes')
set(gca, 'FontSize', 24)
% note that log2(0) == -inf, and that log2(1)==0

In [ ]:
% going back to the above notion that entropy is maximial
% when the outcome is completely uncertain (e.g. a fair coin), then we can
% start to develop an intuition about expressing entropy in terms of the
% probability that some event will occur, denoted P(X). Lets take a more
% complex case - lets say that we have a slot machine that has two wheels on it
% and the first wheel can take one of N states and the second can take one of M states 
% when we pull the handle. Considering just the first wheel, the possible outcomes are
% {x1....xn}, and if each outcome is equally likely, then p(xi)= 1/n. 
% So, for example, if n = 16, then the total entropy of wheel 1 can be represented 
% by 4 bits of information(four binary numbers). 

log2(16)

% now what about the second wheel? It can take on M states. So inutively
% the total number of possible outcomes for both wheels is N*M. The
% uncertainty of the outcome in this case is then:
n = 16;
m = 16;
log2(n*m)

% which recall is equal to 
log2(n) + log2(m) 

% (which gives us a nice way to account for the
% probability of outcomes across multiple variables via summation! why is this 
% computationally convienent, esp on digital computers?)

In [ ]:
% so in this case we need 8 bits of information to specify all possible
% outcomes. Now lets consider each possible outcome in isolation. what is
% the uncertainty associated with each event? its the probability of that
% event occuring, which is p(xi) = 1/n. So in terms of bits, we have :
n = 16;
-log2(1/n) % why negative when dealing with probabilities? log2(1)-log2(n)

In [ ]:
%% AVERAGE UNCERTAINTY what we'd like though is a way to assess the average uncertainty of a
% particular outcome across all possible outcomes. How would you do that?
% Intuitively, you'd take the uncertainty of each outcome (-log2(p(xi))
% and weight it by the probabiltiy that the event will actually 
% occur, like this (where
% entropy is denoted, by convention, as H):

%H = -sum_over_all_i( p(xi) * log2(p(xi)) )
n = 16;
H = 0;
for i=1:n
    H = H + -( (1/n) * log2(1/n) );
end
H

In [ ]:
% what does H, the average uncertaintly, equal when all events are equally likely?
% it equals the uncertaintly of each event, which it should. And note that
% this framework would let you compute the average uncertainty (entropy)
% across both slot machine wheels just as easily by looping over all n an m:
n = 16;
m = 16;
H = 0;
for i=1:n
    H = H + -( (1/n) * log2(1/n) );
end
for i=1:m
    H = H + -( (1/m) * log2(1/m) );
end
H  


In [ ]:
% what about when all events are not equally likely, which is the more
% interesting case to consider.
n = 16;
px = rand(1,n);
px = px./sum(px); % convert the random numbers to probabilities
plot(px)
H = 0;
for i=1:n
    H = H + -( px(i) * log2(px(i)) );
end
H

% note that the entropy is lower because we've moved away from the point where
% everything is maximally unpredictable (i.e. everything is equally
% likely).
% This demonstrates a principle that relates probability
% distributions to entropy: uniform probability distributions have maximum
% entropy, and non-uniform distributions will generally have less entropy

% SUMMARY - entropy is a measure of uncertainty, and as uncertainty goes up
% (maximized when all outcomes are equally likely) then the information
% gained by making a measurement goes up. In other words: if you know the
% oucome in advance (e.g. a coin with two heads) then p(tails) = 0 and there is no
% uncertainty, entropy is 0, and no reduction in uncertainty will be gained by flipping
% the coin. If you have a fair coin, then p(head)==p(tail) and entropy will
% be maximum and you will greatly reduce uncertainty by making the
% measurement (in this case, you will fully disambiguate the outcome,
% gaining 1 bit of information where the total uncertainty is 1 bit - so
% you get the maximum amount of information). 

In [ ]:
n = 2;
ph = .001:.001:.999;  % vary the probability of heads
H = zeros(1,numel(ph));
for j=1:numel(ph)
    px(1) = ph(j);   % p(heads)
    px(2) = 1-px(1); % p(tails))
    for i=1:n
        H(j) = H(j) + -( px(i) * log2(px(i)) );
    end
end
figure(1), hold on
plot(ph, H, 'LineWidth', 2)
set(gca, 'FontSize', 20)
xlabel('Probability of heads (coin bias)')
ylabel('entropy (bits)')
plot([.5 .5], get(gca, 'YLim'), 'k--')

% entropy is maximized with maximum uncertainty and will max out at
% log2(n)!

%% PART III: Mutual information (MI). MI is a measure of how much knowing 
% about 1 variable tells you about the state of another variable. Putting
% aside entropy and measures of uncertainty/variance for a minute, here is the 
% intuition. Suppose you have two variables that are completely unrelated
% to each other: measuring one variable will tell you nothing about the
% state of the other variable. In contrast, if you have two variables that
% are perfectly correlated, then measureing one variable will tell you
% everything about the state of the other. In this special (unusual) case,
% the mutual information will be equal to the entropy of either variable
% alone (that is: the information gained by measuring one variable will be
% equal to the information gained by measuring both)
%
% to put this back in terms of entropy: lets say we have two variables, X
% and Y. If we want to assess the MI between X and Y, then the intuition is
% that we need to know the following difference score:
%
% (total entropy of X) -  (entropy of X given that we know Y). 
%
% In other words, how much is uncertainty about X REDUCED when we measure Y?
% That is the MI between the two variables. And it leads to the definition
% of MI:
%
% MI = H(X) - H(X|Y)  where H(X) is the entropy of X, and H(X|Y) is the
% conditional entropy of X given that we've measured Y - it the average
% entropy of X across all values of Y
% Going back to our
% examples above - suppose X and Y are unrelated (independent):
%
% measuring Y doesn't tell you anything about X, so 
% H(X) == H(X|Y), so MI = H(X) - H(X) = 0
%
% On the other hand, if the two variables are perfectly related,
% then knowing Y will not give us any reduction in uncertainty about X, so:
% 
% H(X|Y) = 0, and MI = H(X) - 0, or MI = H(X)
%
%
% Lets do a few examples - first generate two sequences that are randomly 
% generated idendently from each other

In [ ]:
# discrete

x = round(rand(1,1000));
y = round(rand(1,1000));

px(1) = sum(x)/numel(x);    % probability that x==1
px(2) = 1-px(1);            % prob that x==0

Hx = -sum( px .* log2(px) );    % do in one line instead of looping using the .* operator

% then entropy of x given y (Hxy).
% 1) For all Y {i=1:numel(Y)}, compute the entropy of X given each Y
% 2) Multiply H(X|Yi) with the probability of each Y (i.e. pyi)
% 3) Sum over all i
% this is the average conditional entropy over all X's 

Hxy=0;
uniquey = unique(y);
uniquex = unique(x);
for i=1:numel(uniquey) % loop over unique elements of y, in this case 0,1
    
    % probability that y==y(i) (prob of each y)
    py = numel(find(y==uniquey(i)))/numel(y);

    % then loop over all possible x's to compute entropy of x given each y
    tmp=0;
    for j=1:numel(uniquex)
        px_y = numel(find(x==uniquex(j) & y==uniquey(i)))/numel(find(y==uniquey(i)));       % e.g. prob x==1 when y==0
        tmp = tmp + (-( px_y .* log2(px_y) ));                                              % entropy      
    end
    
    % then tally up entropy of x given specific y times probability of that
    % y (py)
    Hxy = Hxy + py*tmp;
end

% then we have everything we need to compute MI, which in this case should
% be ~0 becuase the variables are completely independent!
MI = Hx - Hxy

In [ ]:
function Hx = entropy(x)

% js - 03032014
% compute entropy of discrete variable x

uniquex = unique(x);

Hx = 0;
for i=1:numel(uniquex)
    px = numel(find(x==uniquex(i)))/numel(x);    % probability that x==uniquex(i)
    if px~=0 % because log2(0) = -inf
        Hx = Hx + (-sum( px * log2(px) ));    
    end
end

In [ ]:
function Hxy = condEntropy(x,y)
% js, 03022014
% conditional entropy of x|y
% i.e. average entropy of x given each y
% Hxy.
% 1) For all Y {i=1:numel(X)}, compute the entropy of X given each Y
% 2) Multiply H(X|Y==i) with the probability of each Y (i.e. pxi)
% 3) Sum over all i

Hxy=0;
uniquey = unique(y);
uniquex = unique(x);
for i=1:numel(uniquey) % loop over unique elements of y
    
    % probability that y==y(i) (prob of each y)
    py = numel(find(y==uniquey(i)))/numel(y);
%     ny(i) = numel(find(y==uniquey(i)));
    
    % then loop over all possible x's to compute entropy of x given each y
    tmp=0;
    for j=1:numel(uniquex)
        px_y = numel(find(x==uniquex(j) & y==uniquey(i)))/numel(find(y==uniquey(i)));     
        if px_y~=0 
            tmp = tmp + ( -sum( px_y .* log2(px_y) ));                                   % entropy      
        end
    end
    
    % then tally up entropy of x given specific y times probability of that
    % y (py)
    Hxy = Hxy + py*tmp;
    
end

In [20]:
function Hxy = mutualInfo(x,y)
% js, 02272014

% first compute the entropy of x
px(1) = sum(x)/numel(x);   % probability that x==1
px(2) = 1-px(1);           % prob that x==0

% Then Hxy.
% 1) For all X {i=1:numel(X)}, compute the entropy of Y given each X
% 2) Multiply H(Y|X==i) with the probability of each X (i.e. pxi)
% 3) Sum over all i

Hxy=0;
for i=1:numel(unique(x)) % loop over unique elements of x, in this case 0,1
    
    % then loop over all possible y's to compute entropy of y given each x
    tmp=0;
    for j=1:numel(unique(y))
        py_x = numel(find(y==y(j) & x==x(i)))/numel(find(x==x(i)));     % e.g. prob y==1 when x==0
        tmp = tmp + (-sum( py_x .* log2(py_x) ));                       % entropy      
    end
    
    % then tally up entropy of y given specific x times px
    Hxy = Hxy + px(i)*tmp;
end

In [ ]:
x = np.random.rand(1000)
y = np.random.rand(1000)


In [4]:
n_bins = 30
def calc_MI(X,Y,bins = np.linspace(-90,90,n_bins+1)):
    bad = np.isnan(X) | np.isnan(Y)
    X = X[~bad]
    Y = Y[~bad]
    
    c_XY = np.histogram2d(X,Y,bins)[0]
    c_X = np.histogram(X,bins)[0]
    c_Y = np.histogram(Y,bins)[0]

    H_X = shan_entropy(c_X)
    H_Y = shan_entropy(c_Y)
    H_XY = shan_entropy(c_XY)
    
    MI = H_X + H_Y - H_XY
    return MI

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

def calc_MI_ds(X,Y,n_samp=2000,n_iter=30,do_shuf=False,ret_all = False):
    MIs = []
    bad = np.isnan(X) | np.isnan(Y)
    X = X[~bad]
    Y = Y[~bad]
    
    for i in range(n_iter):
        if do_shuf:
            np.random.shuffle(X)
        samp = np.random.choice(len(X),n_samp,replace=False)
        MIs.append(calc_MI(X[samp],Y[samp]))
    if ret_all:
        return MIs
    return (np.mean(MIs), np.std(MIs))

In [18]:
x = np.random.rand(1000)*60
y = np.random.rand(1000)*60
n_bins = 600

mi = calc_MI(X=x,Y=y,bins = np.linspace(-90,90,n_bins+1))
print(mi)

5.07719825221
